In [227]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Transportation with Capacity Constraint

In addition to the supply and demand constraints sometimes transportation problems can have capacity restrictions such that each route will have a maximum number of units that may be shipped along that route. (Similar to worker availability in the worker scheduling problem.)

For this problem there are 3 warehouses that ship the same product to 21 stores.  The supply at the warehouses, demand at the stores, unit shipping costs, and route capacities are stored in the `transpSingle.xlsx` file in the `data` folder.  Use the `openpyxl` package to read the problem data into lists and nested lists.  Once you have the problem data converted into dictionaries, then adapt the transportation code in the lesson to minimize the total shipping cost.  You'll need to add the extra capacity constraint to give an upper bound to the number of units shipped on each route.  Your code should also use Technique 3 to eliminate unneeded decision variables.

To get you started the code cell below shows how to use `openpyxl` to read ranges from the Excel workbook.  For full credit you should display the output in a nicely formatted manner (you could use a pandas data frame to display the transported amounts).

<font color = "blue"> *** 10 points -  finish the code in the cell below *** (don't delete this cell) </font>

In [103]:
# using openpyxl
from openpyxl import load_workbook
wb = load_workbook(filename='data/transp_prob_1.xlsx', data_only=True)
sheet = wb.active

# specify upper left and lower right cells, returns a list or list of lists representing rows
def read_range(sheet, begin, end):
    table = sheet[begin:end]
    height = len(table)
    width = len(table[0])
    if height == 1 or width == 1:
        # for a single row or column produce a list
        tmp = [cell.value for row in table for cell in row]
    else:
        # for an array of cells produces a list of row lists
        tmp = [[cell.value for cell in row] for row in table]
    return (tmp)

# finish reading the data
warehouses = read_range(sheet, 'A3', 'A5')
stores = read_range(sheet, 'B3', 'B22')
routes = [(w,s) for [w,s] in read_range(sheet,'D3','E31')]
capacity_dict = {(w,s):cap for [w,s,cap,cost] in read_range(sheet,'D3','G31')}
cost_dict = {(w,s):cost for [w,s,cap,cost] in read_range(sheet,'D3','G31')}
supply_dict = {w:q for [w,q] in read_range(sheet,'I3','J5')}
demand_dict = {s:q for [s,q] in read_range(sheet,'L3','M22')}

# # throw an error if total supply and demand do not match

from pyomo.environ import *

# define model
model = ConcreteModel("prob1")

# set vars
model.transp = Var(routes, domain=NonNegativeReals)

# objective function
model.total_cost = Objective(expr=sum(cost_dict[w,s] * model.transp[w,s] for (w,s) in routes), sense=minimize)

# supply constraint
model.supply_ct = ConstraintList()
for w in warehouses:
    model.supply_ct.add(sum(model.transp[w,s] for s in stores if (w,s) in routes) <= supply_dict[w] )

# demand constraint    
model.demand_ct = ConstraintList()
for s in stores:
    model.demand_ct.add(sum(model.transp[w,s] for w in warehouses if (w,s) in routes) == demand_dict[s] )

# capacity constraint 
model.capacity_ct = ConstraintList()
for (w,s) in routes:
    model.capacity_ct.add(
    model.transp[w, s] <= capacity_dict[w, s])
    model.capacity_ct.add(
    model.transp[w, s] >= 0)  

# solve and display
solver = SolverFactory('glpk')
solver.solve(model)

# # convert model.hrs into a Pandas data frame for nicer display
import pandas as pd
table = pd.DataFrame(0, index=warehouses, columns=stores)
for (w, s) in routes:
    table.loc[w, s] = model.transp[w, s].value

# display
import babel.numbers as numbers  # needed to display as currency
print("The minimum total transportation cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))

from IPython.display import display
display(table)

The minimum total transportation cost =  $29,827.00


,sA,sB,sC,sD,sE,sF,sG,sH,sI,sJ,sK,sL,sM,sN,sO,sP,sQ,sR,sS,sT
wA,173.0,103.0,110.0,121.0,148.0,145.0,100.0,23.0,100.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wB,0.0,0.0,0.0,0.0,0.0,0.0,54.0,100.0,65.0,92.0,175.0,82.0,100.0,100.0,32.0,100.0,0.0,0.0,0.0,0.0
wC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83.0,84.0,65.0,100.0,83.0,146.0,142.0,175.0,122.0


# Shipping Wood to Market

This problem is Case 9.1 on page 370 of the textbook.

Alabama Atlantic is a lumber company that has three sources of wood and five markets to be supplied. The annual availability of wood at sources 1, 2, and 3 is 15, 20, and 15 million board feet, respectively. The amount that can be sold annually at markets 1, 2, 3, 4, and 5 is 11, 12, 9, 10, and 8 million board feet, respectively.  In the past the company has shipped the wood by train. However, because shipping costs have been increasing, the alternative of using ships to make some of the deliveries is being investigated. This alternative would require the company to invest in some ships. Except for these investment costs, the shipping costs in thousands of dollars per million board feet by rail and by water (when feasible) would be the following for each route:

<img src="images/wood_table_1.png" width="600">

The capital investment (in thousands of dollars) in ships required for each million board feet to be transported annually by ship along each route is given as follows:

<img src = "images/wood_table_2.png" width="600">

Considering the expected useful life of the ships and the time value of money, the equivalent uniform annual cost of these investments is one-tenth the amount given in the table. The objective is to determine the overall shipping plan that minimizes the total equivalent uniform annual cost (including shipping costs).

You are the head of the team that has been assigned the task of determining this shipping plan for each of the following three options.
* *Option 1:* Continue shipping exclusively by rail.
* *Option 2:* Switch to shipping exclusively by water (except where only rail is feasible).
* *Option 3:* Ship by either rail or water, depending on which is less expensive for the particular route.

Present your results for each option. Compare.  You should write a short summary of the results and include a recommendation for the best option.  You could modularize your code, though this isn't required, by writing a function that takes the cost array as input and outputs the total cost and transportation amounts.  You could then call the function three times for each of the three options.

<font color = "blue"> *** 15 points -  answer in cells below *** (don't delete this cell) </font>

In [141]:
import numpy as np
sources = (1,2,3)
markets = (1,2,3,4,5)
routes_rail = [(s,m) for s in sources for m in markets]
routes_ship = routes_rail.copy()
# routes_ship.remove((1,4))
# routes_ship.remove((3,1))
supply_dict = {1: 15, 2: 20, 3:15}
demand_dict = {1: 11, 2: 12, 3: 9, 4: 10, 5: 8}
costs_rail = np.multiply((61,72,45,55,66,69,78,60,49,56,59,66,63,61,47),1000).tolist()
costs_ship = np.multiply((31,38,24,1000,35,36,43,28,24,31,1000,33,36,32,26),1000).tolist()
costs_ship_inv = np.multiply((275, 303, 238, 10000, 285, 293, 318, 270, 250, 265, 10000, 283, 275, 268, 240),100).tolist()
route_costs_rail = {(s,m):costs_rail[i] for i, (s,m) in enumerate(routes_rail)}
route_costs_ship = {(s,m):costs_ship[i]+costs_ship_inv[i] for i, (s,m) in enumerate(routes_ship)}
route_costs_all = {route:min((cost_rail, cost_ship)) for route, (cost_rail, cost_ship) in zip(routes_rail,zip(route_costs_rail.values(),route_costs_ship.values()))}
from pyomo.environ import *

# define model



def min_cost_for_route(route_type):
    if route_type == 'rail':
        routes = routes_rail
        route_costs = route_costs_rail
        option = 1

    elif route_type == 'ship':
        routes = routes_ship
        route_costs = route_costs_ship
        option = 2
        
    elif route_type == 'both':
        routes = routes_rail
        route_costs = route_costs_all
        option = 3
        
    model = ConcreteModel("prob2")
    
    # set vars
    model.transp = Var(routes, domain=NonNegativeReals)

    # objective function
    model.total_cost = Objective(expr=sum(route_costs[s,m] * model.transp[s,m] for (s,m) in routes), sense=minimize)

    # supply constraint
    model.supply_ct = ConstraintList()
    for s in sources:
        model.supply_ct.add(sum(model.transp[s,m] for m in markets if (s,m) in routes) <= supply_dict[s] )

    # demand constraint    
    model.demand_ct = ConstraintList()
    for m in markets:
        model.demand_ct.add(sum(model.transp[s,m] for s in sources if (s,m) in routes) == demand_dict[m] )

    # solve and display
    solver = SolverFactory('glpk')
    solver.solve(model)

    # # convert model.hrs into a Pandas data frame for nicer display
    import pandas as pd
    table = pd.DataFrame(0, index=sources, columns=markets)
    for (s, m) in routes:
        table.loc[s, m] = model.transp[s, m].value
        
    table['Total'] = table.apply(np.sum, axis=1)
    table.loc['Total']= table.sum()

    # display
    import babel.numbers as numbers  # needed to display as currency
    print(f"Option {option} - The minimum total transportation cost for {route_type} = ",
          numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))

    from IPython.display import display
    display(table)

min_cost_for_route("rail")
min_cost_for_route("ship")    
min_cost_for_route("both")

Option 1 - The minimum total transportation cost for rail =  $2,816,000.00


,1,2,3,4,5,Total
1,6.0,0.0,9.0,0.0,0.0,15.0
2,2.0,0.0,0.0,10.0,8.0,20.0
3,3.0,12.0,0.0,0.0,0.0,15.0
Total,11.0,12.0,9.0,10.0,8.0,50.0


Option 2 - The minimum total transportation cost for ship =  $2,770,800.00


,1,2,3,4,5,Total
1,6.0,0.0,9.0,0.0,0.0,15.0
2,5.0,0.0,0.0,10.0,5.0,20.0
3,0.0,12.0,0.0,0.0,3.0,15.0
Total,11.0,12.0,9.0,10.0,8.0,50.0


Option 3 - The minimum total transportation cost for both =  $2,729,100.00


,1,2,3,4,5,Total
1,6.0,0.0,9.0,0.0,0.0,15.0
2,5.0,0.0,0.0,10.0,5.0,20.0
3,0.0,12.0,0.0,0.0,3.0,15.0
Total,11.0,12.0,9.0,10.0,8.0,50.0


Summary:

The best option is option 3, at which the total transportation cost is minimized. This makes sense, because the third model selects the least expensive shipping mode for each route.

# A more complex transportation problem

Three different products pA, pB, and, pC, must be transported from factories to warehouses and then from the warehouses to stores.  The cost per unit varies along different route segments.  Your job is to minimize the total transportation cost.  The constraints are as follows:
* the amount of each product at each factory must match the supply amount
* the amount of each product delivered to each store must match the demand amount
* the amount of each product shipped to each warehouse must be the same as the amount of each product shipped from each warehouse
* the total (summed) amount of all products at each warehouse must be $\leq$ `MaxStorage`
* the total (summed) amount of all products shipped from each factory to each warehouse must be $\leq$ `CapacityFW`
* the total (summed) amount of all products shipped from each warehouse to each store must be $\leq$ `CapacityWS`

The data for this problem is in `transp_prob3.xlsx`.  Since many routes aren't used you'll need to use one of the techniques in the lesson to make sure that no product is shipped along an unused route. 

Your solution should display the total transportation cost as well as the amount transported from both factories to warehouses and from warehouses to stores.  The transport multiple products example in the lesson is a good place to start.  Note that you'll need two "arrays" of decision variables for the two sets of connections.

<font color = "blue"> *** 15 points -  answer in cell below *** (don't delete this cell) </font>

In [246]:
# using openpyxl
from openpyxl import load_workbook
wb = load_workbook(filename='data/transp_prob3.xlsx', data_only=True)
sheet = wb.active

# specify upper left and lower right cells, returns a list or list of lists representing rows
def read_range(sheet, begin, end):
    table = sheet[begin:end]
    height = len(table)
    width = len(table[0])
    if height == 1 or width == 1:
        # for a single row or column produce a list
        tmp = [cell.value for row in table for cell in row]
    else:
        # for an array of cells produces a list of row lists
        tmp = [[cell.value for cell in row] for row in table]
    return (tmp)

# finish reading the data
factories = read_range(sheet, 'A2', 'A6')
warehouses = read_range(sheet, 'B2', 'B11')
stores = read_range(sheet, 'C2', 'C21')
products = read_range(sheet, 'D2', 'D6')
capacityfw = 1000
capacityws = 800
maxstorage = 2300
fw_routes = {(p, f, w) for [p, f, w] in read_range(sheet, 'G3', 'I71')}
ws_routes = {(p, w, s) for [p, w, s] in read_range(sheet, 'L3', 'N202')}
supply_dict = {(p,f):q for [p, f, q] in read_range(sheet,'Q3','S15')}
demand_dict = {(p,s):q for [p, s, q] in read_range(sheet,'U3','W102')}
fw_cost_dict = {(p, f, w):c for [p, f, w, c] in read_range(sheet, 'G3', 'J71')}
ws_cost_dict = {(p, w, s):c for [p, w, s, c] in read_range(sheet, 'L3', 'O202')}

# model
from pyomo.environ import *

# define model
model = ConcreteModel("prob3")

# set vars
model.transp_fw = Var(fw_routes, domain=NonNegativeReals)
model.transp_ws = Var(ws_routes, domain=NonNegativeReals)



# objective function
model.total_cost = Objective(expr=sum(fw_cost_dict[p,f,w] * model.transp_fw[p,f,w] for (p,f,w) in fw_routes)+
                             sum(ws_cost_dict[p,w,s] * model.transp_ws[p,w,s] for (p,w,s) in ws_routes), 
                             sense=minimize)

# max storage constraint
model.max_storage = ConstraintList()

# supply constraint
model.supply_ct = ConstraintList()
for p in products:
    for f in factories:
        if (p,f) in supply_dict:
            model.supply_ct.add(sum(model.transp_fw[p,f,w] for w in warehouses if (p,f,w) in fw_routes) 
                                == supply_dict[p,f])
    model.max_storage.add(sum(model.transp_fw[p,f,w] for w in warehouses for f in factories if (p,f,w) in fw_routes and (p,f) in supply_dict)-
                         sum(model.transp_ws[p,w,s] for w in warehouses for s in stores if (p,w,s) in ws_routes and (p,s) in demand_dict)
                                <= maxstorage)

# capacity fw constraint
model.capacity_fw = ConstraintList()
for (p, f, w) in fw_routes:
    if (p,f) in supply_dict:
        model.capacity_fw.add(model.transp_fw[p,f,w] <= capacityfw)

# capacity ws constraint
model.capacity_ws = ConstraintList()
for (p, w, s) in ws_routes:
    if (p,s) in supply_dict:
        model.capacity_ws.add(model.transp_ws[p,w,s] <= capacityws)

# demand constraint    
model.demand_ct = ConstraintList()
for p in products:
    for s in stores:
        if (p,s) in demand_dict:
            model.demand_ct.add(sum(model.transp_ws[p,w,s] for w in warehouses if (p,w,s) in ws_routes) 
                                == demand_dict[p,s])

# equivalance constraint    
model.equivalence_dict = ConstraintList()
for p in products:
    for w in warehouses:
        model.equivalence_dict.add(sum(model.transp_ws[p,w,s] for s in stores if (p,w,s) in ws_routes and (p,s) in demand_dict) 
                                   == sum(model.transp_fw[p,f,w] for f in factories if (p,f,w) in fw_routes and (p,f) in supply_dict))

# solve and display
solver = SolverFactory('glpk')
solver.solve(model)

import pandas as pd

# plot FW table
cols_fw = [products, factories]
idx_fw = pd.MultiIndex.from_product(cols_fw, names=['products','factories'])
fw_table = pd.DataFrame(0, index=idx_fw, columns=warehouses)

for (p, f, w) in fw_routes:
    fw_table.loc[(p, f), w] = model.transp_fw[p, f, w].value
    
fw_table['Total'] = fw_table.apply(np.sum, axis=1)
fw_table.loc['Total']= fw_table.sum()

# plot WS table
cols_ws = [products, warehouses]
idx_ws = pd.MultiIndex.from_product(cols_ws, names=['products','warehouses'])
ws_table = pd.DataFrame(0, index=idx_ws, columns=stores)

for (p, w, s) in ws_routes:
    ws_table.loc[(p, w), s] = model.transp_ws[p, w, s].value

ws_table['Total'] = ws_table.apply(np.sum, axis=1)
ws_table.loc['Total']= ws_table.sum()


# display results
import babel.numbers as numbers  # needed to display as currency
print("The minimum total transportation cost = ",
      numbers.format_currency(model.total_cost(), 'USD', locale='en_US'))

from IPython.display import display
display(fw_table)
display(ws_table)

The minimum total transportation cost =  $44,860.00


,wA,wB,wC,wD,wE,wF,wG,wH,wI,wJ,Total
"(pA, fA)",240.0,220.0,10.0,230.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0
"(pA, fB)",0.0,0.0,120.0,0.0,440.0,0.0,0.0,0.0,0.0,0.0,560.0
"(pA, fC)",0.0,0.0,0.0,0.0,110.0,0.0,170.0,180.0,120.0,220.0,800.0
"(pA, fD)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(pA, fE)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(pB, fA)",260.0,200.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0
"(pB, fB)",0.0,0.0,150.0,170.0,590.0,90.0,0.0,0.0,0.0,0.0,1000.0
"(pB, fC)",0.0,0.0,0.0,0.0,0.0,0.0,60.0,160.0,140.0,240.0,600.0
"(pB, fD)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(pB, fE)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,sA,sB,sC,sD,sE,sF,sG,sH,sI,sJ,...,sL,sM,sN,sO,sP,sQ,sR,sS,sT,Total
"(pA, wA)",150.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,240.0
"(pA, wB)",0.0,0.0,80.0,140.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,220.0
"(pA, wC)",0.0,0.0,0.0,0.0,60.0,70.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,130.0
"(pA, wD)",0.0,0.0,0.0,0.0,0.0,0.0,110.0,120.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,230.0
"(pA, wE)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0,200.0,...,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,550.0
"(pA, wF)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"(pA, wG)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,60.0,110.0,0.0,0.0,0.0,0.0,0.0,0.0,170.0
"(pA, wH)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,120.0,60.0,0.0,0.0,0.0,0.0,180.0
"(pA, wI)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,70.0,50.0,0.0,0.0,120.0
"(pA, wJ)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,120.0,220.0


# Assignment Problem 9.3-4 from the textbook

The coach of an age group swim team needs to assign swimmers to a 200-yard medley relay team to send to the Junior Olympics. Since most of his best swimmers are very fast in more than one stroke, it is not clear which swimmer should be assigned to each of the four strokes. The five fastest swimmers and the best times (in seconds) they have achieved in each of the strokes (for 50 yards) are

<img src = "images/swim.png" width="600">

The coach wishes to determine how to assign four swimmers to the four different strokes to minimize the sum of the corresponding best times.  

Treat this as an assignment problem and use Pyomo to find the optimal solution.  For full credit display nicely formatted output.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [ ]:
# solution can go in this code cell with output below.